this notebook is used to study velocity dispersion maps and velocity dispersion distribution

In [ ]:
import time
from multiprocessing import Pool
import glob
import os
from natsort import natsorted
from postprocessing import *

        
def weighted_std(values, weights=None):
    """
    Return the weighted standard deviation.

    values, weights -- NumPy ndarrays with the same shape.
    """
    average = np.average(values, weights=weights)
    # Fast and numerically precise:
    variance = np.average((values-average)**2, weights=weights)
    return np.sqrt(variance)

def sigma(x):
    """
    Calculate velocity dispersion for x and y_grid (global parameter...)
    """
    result = []
    
    if projection == 'xy':
        x0 = 0
        x1 = 1
        x2 = 2
    elif projection == 'xz':
        x0 = 0
        x1 = 2
        x2 = 1   
    elif projection == 'yz':
        x0 = 2
        x1 = 1
        x2 = 0
    temperatures = get_temp(output_directory + filename, 5/3)
    
    
    for y in y_grid:
        mask = ((snap_data['PartType0/Coordinates'][:][:, x0] > x) & 
                (snap_data['PartType0/Coordinates'][:][:, x0] < x + dx) &
                (snap_data['PartType0/Coordinates'][:][:, x1] > y) &
                (snap_data['PartType0/Coordinates'][:][:, x1] < y + dy) &
                (snap_data['PartType0/Coordinates'][:][:, x2] > 500) &
                (snap_data['PartType0/Coordinates'][:][:, x2] < 1500) &
                (temperatures < 1e5))
        try: 
            velocity_dispersion = weighted_std(snap_data['PartType0/Velocities'][:][:, x2][mask], weights=densities[mask] **2)
        except: 
            velocity_dispersion = np.nan
        result.append([x, y, velocity_dispersion])
    return result

In [ ]:
mpl.rcParams['figure.dpi']= 200

In [ ]:
N = 51
x_grid = np.linspace(500, 1500, N)
y_grid = np.linspace(500, 1500, N)
dx, dy = np.diff(x_grid)[0], np.diff(y_grid)[0]

In [ ]:
density = '100'
mach    = '4'
jet     = '42'
stage   = '12'

In [ ]:
#simulation_directory = f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_drive_center_d{density}_m{mach}/jet{jet}_{stage}'
simulation_directory = f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_drive_center_d{density}_m{mach}/turb_alter_{stage}'

output_directory = simulation_directory+"/output/"
figures_directory = simulation_directory + "/output/figures/"

## observational resolution

In [ ]:
0.044 * 300000/70 * np.sin(np.deg2rad(0.5 /60 /60)) * 1e6 # in pc resolution from observations?!

## high res maps

In [ ]:
i_file = 15#-1
while True:
    i_file += 1
    try:
        filename = "snap_%03d.hdf5" % (i_file)
        snap_data = h5py.File(output_directory + filename, "r")
    except:
        break
    densities = snap_data['PartType0/Density'][:]
    projection = 'xy'
    starttime = time.time()
    pool = Pool(32)
    sigma_result = pool.map(sigma, x_grid)
    pool.close()
    endtime = time.time()
    print(f"Time taken {endtime-starttime} seconds for snapshot {i_file}")
    sigma_result = np.array(sigma_result)
    sigma_result = sigma_result.reshape(N * N, 3)
    sigma_result = sigma_result.T
    
    if projection == 'xy':
        xlabel = 'x [pc]'
        ylabel = 'y [pc]'
        clabel = r'$\sigma_z$ [km / s]'
    elif projection == 'xz':
        xlabel = 'x [pc]'
        ylabel = 'z [pc]'  
        clabel = r'$\sigma_y$ [km / s]'
    elif projection == 'yz':
        xlabel = 'y [pc]'
        ylabel = 'z [pc]'
        clabel = r'$\sigma_x$ [km / s]'
        
    fig, ax = plt.subplots(figsize=(8, 5))
    c = ax.scatter(sigma_result[0], sigma_result[1], c=sigma_result[2], cmap='plasma', norm=colors.LogNorm(vmin=10, vmax=80), s=25, marker='s')
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_aspect('1')
    ax.set_xlim(500, 1500)
    ax.set_ylim(500, 1500)
    ax.scatter(1000, 1000, c='white', s=10)
    plt.colorbar(c, ax=ax, label=clabel)
    ax.set_title("t=%.2f Myr"%(get_time_from_snap(snap_data) * unit_time_in_megayr))
    plt.savefig(figures_directory + f'veldispersion_1kpc_{projection}_{i_file}.png', dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
from PIL import Image
import glob
from natsort import natsorted

# make gif
#--------------------------
def crop_img(im):
    width, height = im.size
    left = 9
    top =  3
    right = width - 3
    bottom = height - 9
    im = im.crop((left, top, right, bottom))
    return im

ifilename = figures_directory + f'/veldispersion_1kpc_{projection}*.png'
ofilename = figures_directory + f'/veldispersion_1kpc_{projection}.gif'
imgs = natsorted(glob.glob(ifilename))

timestep=4

frames = []
for i in imgs:
    new_frame = Image.open(i)
    frames.append(crop_img(new_frame))

frames[0].save(ofilename, format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=len(imgs) * timestep, loop=0)

## velocity distribution per pixel:

In [ ]:
# i_file = 0
# filename = "snap_%03d.hdf5" % (i_file)
# snap_data = h5py.File(output_directory + filename, "r")
# densities = snap_data['PartType0/Density'][:]

# projection = 'xy'

# starttime = time.time()
# pool = Pool(32)
# sigma_result = pool.map(sigma, x_grid)
# pool.close()
# endtime = time.time()
# print(f"Time taken {endtime-starttime} seconds")

# sigma_result = np.array(sigma_result)
# sigma_result = sigma_result.reshape(N * N, 3)
# sigma_result = sigma_result.T

# if projection == 'xy':
#     xlabel = 'x [pc]'
#     ylabel = 'y [pc]'
#     clabel = r'$\sigma_z$ [km / s]'
# elif projection == 'xz':
#     xlabel = 'x [pc]'
#     ylabel = 'z [pc]'  
#     clabel = r'$\sigma_y$ [km / s]'
# elif projection == 'yz':
#     xlabel = 'y [pc]'
#     ylabel = 'z [pc]'
#     clabel = r'$\sigma_x$ [km / s]'

# fig, ax = plt.subplots(figsize=(8, 5))
# c = ax.scatter(sigma_result[0], sigma_result[1], c=sigma_result[2], cmap='plasma', norm=colors.LogNorm(vmin=10, vmax=80), s=25, marker='s')
# ax.set_xlabel(xlabel)
# ax.set_ylabel(ylabel)
# ax.set_aspect('1')
# ax.set_xlim(500, 1500)
# ax.set_ylim(500, 1500)
# ax.scatter(1000, 1000, c='white', s=10)
# plt.colorbar(c, ax=ax, label=clabel)
# plt.savefig(figures_directory + f'veldispersion_{projection}_{i_file}.png', dpi=300, bbox_inches='tight')

In [ ]:
x = x_grid[N // 4]
y = y_grid[N // 2]

In [ ]:
result = []
temperatures = get_temp(output_directory + filename, 5/3)

mask = ((snap_data['PartType0/Coordinates'][:][:,0] > x) & 
        (snap_data['PartType0/Coordinates'][:][:,0] < x + dx) &
        (snap_data['PartType0/Coordinates'][:][:,1] > y) &
        (snap_data['PartType0/Coordinates'][:][:,1] < y + dy) &
#         (snap_data['PartType0/Coordinates'][:][:,2] > 0) &
        (temperatures < 10 ** 4))

velocities = snap_data['PartType0/Velocities'][:][:,2][mask]
masses     = snap_data['PartType0/Masses'][:][mask]

In [ ]:
mean = np.mean(velocities)
std = weighted_std(velocities, weights=densities[mask] **2)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(11, 4))
c = ax[0].scatter(sigma_result[0], sigma_result[1], c=sigma_result[2], cmap='plasma', norm=colors.LogNorm(vmin=10, vmax=80), s=15, marker='s')
ax[0].set_xlabel(xlabel)
ax[0].set_ylabel(ylabel)
ax[0].set_aspect('1')
ax[0].set_xlim(500, 1500)
ax[0].set_ylim(500, 1500)
ax[0].scatter(500, 500, c='white', s=10, marker='o')
ax[0].scatter(x, y, c='white', marker='s', s=10)
cb = plt.colorbar(c, ax=ax[0])
cb.set_label(label=r'$\sigma_z$ [km / s]', labelpad=-10)

ax[1].hist(velocities, bins=30, weights=masses)
ax[1].set_xlabel('$v_z$ [km / s]')
ax[1].set_ylabel('number')
ax[1].set_xlim(-300, 300)
ax[1].errorbar(mean, 5000, xerr=std, ms=5, capsize=5, fmt='o', color='black')
plt.subplots_adjust(wspace=0.3)

### dispersion in the whole box as a function of time

In [ ]:
density = '10'
mach    = '8'
jet     = '38'
stage   = '15'

#simulation_directory = f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/jet{jet}_{stage}'
simulation_directory = f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/turb'

output_directory = simulation_directory+"/output/"
figures_directory = simulation_directory + "/output/figures/"

In [ ]:
stds = []
bins = np.linspace(-300, 300, 51)

In [ ]:
i_file = -1

while True:
    i_file += 1
    filename = "snap_%03d.hdf5" % (i_file)
    try:
        snap_data = h5py.File(output_directory + filename, "r")
    except:
        break
    
    temperatures = get_temp(output_directory + filename, 5/3)
    mask = (temperatures < 10 ** 4.5)
    
    velocities = snap_data['PartType0/Velocities'][:][:,2][mask]
    coordinates = snap_data['PartType0/Coordinates'][:][mask]
    masses     = snap_data['PartType0/Masses'][:][mask]
    densities     = snap_data['PartType0/Masses'][:][mask]
    
    mask_outside = (((coordinates[:, 0] < 500) | 
                    (coordinates[:, 0] > 1500)) &
                    ((coordinates[:, 1] < 500) |
                    (coordinates[:, 1] > 1500)) &
                    ((coordinates[:, 2] < 500) |
                    (coordinates[:, 2] > 1500)))
    
    mask_inside =  ((coordinates[:, 0] > 500) & 
                    (coordinates[:, 0] < 1500) &
                    (coordinates[:, 1] > 500) &
                    (coordinates[:, 1] < 1500) &
                    (coordinates[:, 2] > 500) &
                    (coordinates[:, 2] < 1500))

    std = weighted_std(velocities[mask_outside], weights=densities[mask_outside] **2)
    
#     if i_file==0 or i_file==7:
#         plt.hist(velocities, weights=masses, bins=bins, alpha=0.5, label=f't={np.round(get_time_from_snap(snap_data), 2)} Myr')
#         plt.xlabel(r'$v_z$ [km / s]')
#         plt.ylabel('mass weighted cells number')
#         plt.xlim(-300, 300)
#         plt.ylim(1, 7e8)
#         plt.legend()
    stds.append([get_time_from_snap(snap_data), std])

In [ ]:
stds_turb = np.array(stds).T

In [ ]:
plt.plot(stds_38[0] - stds_38[0][0], stds_38[1], label='jet power 38')
plt.plot(stds_40[0] - stds_40[0][0], stds_40[1], label='jet power 40')
plt.plot(stds_43[0] - stds_43[0][0], stds_43[1], label='jet power 43')
plt.plot(stds_turb[0] - stds_turb[0][0], stds_turb[1], label='turb')
plt.ylim(40, 110)
plt.xlim(0, 25)
plt.xlabel('time [Myr]')
plt.ylabel(r'$\sigma_z$ [km / s]')

plt.legend(fontsize=9)

In [ ]:
plt.plot(stds_42[0], stds_42[1], label='jet power 1e42')
plt.plot(stds_44[0], stds_44[1], label='jet power 1e44')
plt.plot(stds_alter[0], stds_alter[1], label='turb box')
plt.ylim(38, 48)
plt.xlabel('time [Myr]')
plt.ylabel(r'$\sigma_z$ [km / s]')
plt.gr
plt.legend()

## radial velocity dispersion

In [ ]:
density = '30'
mach    = '8'
jet     = '43'
start   = '15'
simulation_directory = str(f'/n/holystore01/LABS/hernquist_lab/Users/borodina/2kpc/turb_drive_center_d{density}_m{mach}/jet{jet}_{start}')
#simulation_directory = str(f'/n/holystore01/LABS/hernquist_lab/Users/borodina/turb_drive_center_d{density}_m{mach}/turb')

output_directory = simulation_directory + "/output/"
figures_directory = simulation_directory + "/output/figures/"


In [ ]:
filename = "snap_%03d.hdf5" % (15)
fig, ax = plt.subplots(figsize=(6,7))
plot_radvelocity(ax, output_directory + filename)

In [ ]:
filename = "snap_%03d.hdf5" % (15)
fig, ax = plt.subplots(figsize=(6,7))
plot_shocks(ax, output_directory + filename)

## radial velocity distribution

In [ ]:
density = '10'
mach    = '8'
jet     = '44'
start   = '12'

simulation_directory = str(f'/n/holystore01/LABS/hernquist_lab/Users/borodina/2kpc/turb_drive_center_d{density}_m{mach}/jet{jet}_{start}')
#simulation_directory = str(f'/n/holystore01/LABS/hernquist_lab/Users/borodina/2kpc/turb_drive_center_d{density}_m{mach}/turb_alter_{start}')

output_directory = simulation_directory + "/output/"
figures_directory = simulation_directory + "/output/figures/"

bins = np.linspace(-1e3, 1e3, 251)

In [ ]:
i_file = -1
while True:
    i_file += 1

    projection = 'radial'
    try:
        filename_jet = "snap_%03d.hdf5" % (i_file)
        snap_data_jet = h5py.File(output_directory + filename_jet, "r")
    except:
        break
    
    x, y, z = snap_data_jet['PartType0/Coordinates'][:].T
    vx, vy, vz = snap_data_jet['PartType0/Velocities'][:].T
    center = snap_data_jet['Header'].attrs['BoxSize'] / 2
    v_r_jet = (vx * (x - center) + vy * (y - center) + vz * (z - center)) / \
                np.sqrt((x - center) ** 2 + (y - center) ** 2 + (z - center) ** 2)

    temperatures_jet = get_temp(output_directory + filename_jet, 5/3)
    masses_jet = snap_data_jet['PartType0/Masses'][:]
    densities_jet = snap_data_jet['PartType0/Density'][:]
    volumes_jet = masses_jet / densities_jet 
    radius_jet = np.sqrt((x - center) ** 2 + (y - center) ** 2 + (z - center) ** 2)

    mask_hot_jet  = ((temperatures_jet > 10 ** 4.5) & (radius_jet > 300) & (radius_jet < 500))
    mask_warm_jet = ((temperatures_jet < 10 ** 4.5) & (radius_jet > 300) & (radius_jet < 500))


    i_file_turb = 0
    projection = 'radial'

    filename_turb = "snap_%03d.hdf5" % (i_file_turb)
    snap_data_turb = h5py.File(output_directory + filename_turb, "r")

    x, y, z = snap_data_turb['PartType0/Coordinates'][:].T
    vx, vy, vz = snap_data_turb['PartType0/Velocities'][:].T
    v_r_turb = (vx * (x - center) + vy * (y - center) + vz * (z - center)) / np.sqrt((x - center) ** 2 + (y - center) ** 2 + (z - center) ** 2)

    temperatures_turb = get_temp(output_directory + filename_turb, 5/3)
    masses_turb = snap_data_turb['PartType0/Masses'][:]
    densities_turb = snap_data_turb['PartType0/Density'][:]
    volumes_turb = masses_turb / densities_turb 
    radius_turb = np.sqrt((x - center) ** 2 + (y - center) ** 2 + (z - center) ** 2)

    mask_hot_turb  = ((temperatures_turb > 10 ** 4.5) & (radius_turb > 300) & (radius_turb < 500))
    mask_warm_turb = ((temperatures_turb < 10 ** 4.5) & (radius_turb > 300) & (radius_turb < 500))

    fig, ax = plt.subplots(figsize=(5.5, 3))
    ax.hist((v_r_turb[mask_hot_turb]), bins=bins, 
             log=True, weights=masses_turb[mask_hot_turb], label='hot gas in initial box', alpha=0.5, color='tab:red')
    ax.hist((v_r_turb[mask_warm_turb]), bins=bins, 
             log=True, weights=masses_turb[mask_warm_turb], label='warm gas in initial box', alpha=0.5, color='tab:blue')


    ax.hist((v_r_jet[mask_hot_jet]), bins=bins, histtype='step', 
             log=True, weights=masses_jet[mask_hot_jet], label='hot gas when jet is on', ec='tab:red',linewidth=1.5)
    ax.hist((v_r_jet[mask_warm_jet]), bins=bins, histtype='step', 
             log=True, weights=masses_jet[mask_warm_jet], label='warm gas when jet is on', ec='tab:blue', linewidth=1.5)


    plt.title(fr't = {np.round(get_time_from_snap(snap_data_jet) * unit_time_in_megayr, 2)} Myr')
    plt.legend(loc='upper right',fontsize=7, fancybox=True, framealpha=0.5)
    ax.set_xlim(-700, 700)
    ax.set_ylim(100, 1e10)
    ax.set_xlabel(r"$v_r$ [km / s]")
    ax.set_ylabel(r'mass-weighted distribution' +'\n' +'of cells $r > 300$ pc')
    
    plt.savefig(figures_directory + f'velocity_distribution_{i_file}.png', dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
from PIL import Image

# make gif
#--------------------------
def crop_img(im):
    width, height = im.size
    left = 9
    top =  3
    right = width - 3
    bottom = height - 9
    im = im.crop((left, top, right, bottom))
    return im

ifilename = figures_directory + '/velocity_distribution*.png'
ofilename = figures_directory + '/velocity_distribution.gif'
imgs = natsorted(glob.glob(ifilename))

timestep=4

frames = []
for i in imgs:
    new_frame = Image.open(i)
    frames.append(crop_img(new_frame))

frames[0].save(ofilename, format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=len(imgs) * timestep, loop=0)